<div style="width: 100%">
    <img style="width: 100%" src="https://storage.googleapis.com/kaggle-competitions/kaggle/23870/logos/header.png"/>
</div>

Huge Shout out to these kernels.

1. Sin's https://www.kaggle.com/underwearfitting/resnet200d-public-benchmark-2xtta-lb0-965/data?scriptVersionId=51087772
2. Ammarali32's https://www.kaggle.com/ammarali32/resnet200d-inference-single-model-lb-96-5/data

Please like their kernels. In this exercise, we are trying to ensemble the two strategies together

<pre>
Logs
Version 10 -  Resnet200d Public Benchmark + Inference Model (.967)
Version 6 -  Resnet200d Public Benchmark + Multi-Head Model (.966)
</pre>

Configuration : Sin's Model

In [ ]:
import pandas as pd
batch_size = 1
image_size = 512
tta = True
submit = True
enet_type = ['resnet200d'] * 5
model_path = ['../input/resnet200d-baseline-benchmark-public/resnet200d_fold0_cv953.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold1_cv955.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold2_cv955.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold3_cv957.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold4_cv954.pth']

Imports

In [ ]:
import os
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import numpy as np
DEBUG = False
import time
import cv2
import PIL.Image
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import albumentations
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pylab import rcParams
import timm
from albumentations import *
from albumentations.pytorch import ToTensorV2
device = torch.device('cuda') if not DEBUG else torch.device('cpu')

Model

In [ ]:
class RANZCRResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d', out_dim=11, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, out_dim)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

Transforms

In [ ]:
transforms_test = albumentations.Compose([
    Resize(image_size, image_size),
    Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ToTensorV2()
])

Dataset

In [ ]:
class RANZCRDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        self.labels = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        img = cv2.imread(row.file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image']
        label = torch.tensor(self.labels[index]).float()
        if self.mode == 'test':
            return img
        else:
            return img, label

In [ ]:
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
test['file_path'] = test.StudyInstanceUID.apply(lambda x: os.path.join('../input/ranzcr-clip-catheter-line-classification/test', f'{x}.jpg'))
target_cols = test.iloc[:, 1:12].columns.tolist()

test_dataset = RANZCRDataset(test, 'test', transform=transforms_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=24)

Utils

In [ ]:
def inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader)
    LOGITS = []
    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            logits = model(x)
            LOGITS.append(logits.cpu())
            PREDS += [logits.sigmoid().detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()
        LOGITS = torch.cat(LOGITS).cpu().numpy()
    return PREDS

def tta_inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader)
    PREDS = []
    LOGITS = []

    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            x = torch.stack([x,x.flip(-1)],0) # hflip
            x = x.view(-1, 3, image_size, image_size)
            logits = model(x)
            logits = logits.view(batch_size, 2, -1).mean(1)
            PREDS += [logits.sigmoid().detach().cpu()]
            LOGITS.append(logits.cpu())
        PREDS = torch.cat(PREDS).cpu().numpy()
        
    return PREDS

Prepare the first Prediction

In [ ]:
if submit:
    test_preds_1 = []
    for i in range(len(enet_type)):
        if enet_type[i] == 'resnet200d':
            print('resnet200d loaded')
            model = RANZCRResNet200D(enet_type[i], out_dim=len(target_cols))
            model = model.to(device)
        model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
        if tta:
            test_preds_1 += [tta_inference_func(test_loader)]
        else:
            test_preds_1 += [inference_func(test_loader)]

In [ ]:
submission = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
submission[target_cols] = np.mean(test_preds_1, axis=0)

Ammarali32's Model

In [ ]:
import os
import sys
sys.path.append('../input/pytorch-images-seresnet')
import timm

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
IMAGE_SIZE = 640
BATCH_SIZE = 128
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'
MODEL_PATH = '../input/resnet200d-public/resnet200d_320_CV9632.pth'
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
def get_transforms():
        return Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(
            ),
            ToTensorV2(),
        ])

In [ ]:
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [ ]:
def inference(models, test_loader, device):
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for model in models:
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

In [ ]:
model = ResNet200D()
model.load_state_dict(torch.load(MODEL_PATH)['model'])
model.eval()
models = [model.to(device)]

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=4 , pin_memory=True)
predictions = inference(models, test_loader, device)

In [ ]:
test[target_cols] = predictions
test[target_cols].head(5)

In [ ]:
submission[target_cols].head(5)

In [ ]:
# DISCUSSION : https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/211194
Final_Submission = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

Final_Submission[target_cols] = (test[target_cols]**0.5 + submission[target_cols]**0.5)/2

In [ ]:
Final_Submission[target_cols].head(5)

In [ ]:
Final_Submission.to_csv("submission.csv", index=False)